# Get All Tweets with #PodRevDay

In [1]:
# Get the data
#!snscrape --jsonl twitter-hashtag "#podrevday since:2020-01-01 until:2020-10-31" > 'data/jan-oct-2020.json'

In [2]:
import pandas as pd
from geotext import GeoText

df_tweets = pd.read_json('data/jan-oct-2020.json', lines=True)

In [3]:
df_tweets.shape

(1255, 21)

In [4]:
df_tweets.head().T

,0,1,2,3,4
url,https://twitter.com/stephfuccio/status/1320470...,https://twitter.com/LizWilcox79/status/1319443...,https://twitter.com/PodRevDay/status/131574234...,https://twitter.com/EarbudsPodCol/status/13153...,https://twitter.com/mindfulness_for/status/131...
date,2020-10-25 21:00:08+00:00,2020-10-23 00:58:20+00:00,2020-10-12 19:53:16+00:00,2020-10-11 17:28:45+00:00,2020-10-09 17:45:47+00:00
content,#PodRevDay is Nov 8th:\nhttps://t.co/eRuEJxABa...,I'm listening to so many great podcasts whilst...,I need this @DishPodcast episode on stress rig...,Pod News\n \nThe first Thurs of each month is ...,"Its #PodRevDay - if youve heard the podcast, w..."
renderedContent,#PodRevDay is Nov 8th:\n1.review a podcast\n2....,I'm listening to so many great podcasts whilst...,I need this @DishPodcast episode on stress rig...,Pod News\n \nThe first Thurs of each month is ...,"Its #PodRevDay - if youve heard the podcast, w..."
id,1320470219463294981,1319443001920221184,1315742349155340289,1315343595818409987,1314623106556850177
user,"{'username': 'stephfuccio', 'displayname': 'St...","{'username': 'LizWilcox79', 'displayname': 'El...","{'username': 'PodRevDay', 'displayname': 'PodR...","{'username': 'EarbudsPodCol', 'displayname': '...","{'username': 'mindfulness_for', 'displayname':..."
outlinks,"[http://1.review, http://3.Read]",[],[],[],[https://podcasts.apple.com/gb/podcast/mindful...
tcooutlinks,"[https://t.co/eRuEJxABaI, https://t.co/my3vCDs...",[],[],[],[https://t.co/55dwUav5ZV]
replyCount,0,0,0,1,0
retweetCount,3,4,1,1,0


# Get User Data

In [5]:
df_tweets.user[0]

{'username': 'stephfuccio',
 'displayname': 'Stephanie Fuccio',
 'id': 153349145,
 'description': 'Freelance Podcast Editor | Podcast Host/Producer of Geopats Language (@geopatslanguage) | Founder of #PodRevDay @podrevday: Currently in #Tirana #Albania',
 'rawDescription': 'Freelance Podcast Editor | Podcast Host/Producer of Geopats Language (@geopatslanguage) | Founder of #PodRevDay @podrevday: Currently in #Tirana #Albania',
 'descriptionUrls': [],
 'verified': False,
 'created': '2010-06-08T09:24:09+00:00',
 'followersCount': 2087,
 'friendsCount': 2205,
 'statusesCount': 16558,
 'favouritesCount': 6107,
 'listedCount': 107,
 'mediaCount': 2189,
 'location': 'Albania',
 'protected': False,
 'linkUrl': 'http://www.stephfuccio.com',
 'linkTcourl': 'https://t.co/U36DU96CmK',
 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1263172564274360322/T6CQl6Ry_normal.jpg',
 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/153349145/1516769927',
 'url': 'https://twitter.com/st

In [6]:
df_tweets['username'] = df_tweets['url'].str.split('/').str[3]

In [7]:
#pd.DataFrame(ser.tolist())
pod_rev_users = pd.DataFrame(df_tweets.user.to_list())

In [8]:
pod_rev_users.head().T

,0,1,2,3,4
username,stephfuccio,LizWilcox79,PodRevDay,EarbudsPodCol,mindfulness_for
displayname,Stephanie Fuccio,Elizabeth Wilcox,PodRevDay,Earbuds Podcast Collective,"Shaun, Mindfulness for Beginners Podcast"
id,153349145,788584258826665984,1258481717431750656,821850926243659776,1220652146649128963
description,Freelance Podcast Editor | Podcast Host/Produc...,"Writer, Avid Role-Player, Amateur Mixologist, ...","Podcast Review Day, a monthly Podcast Listener...",We send a weekly email with a theme and 5 pods...,Join the weekly #podcast designed to help you ...
rawDescription,Freelance Podcast Editor | Podcast Host/Produc...,"Writer, Avid Role-Player, Amateur Mixologist, ...","Podcast Review Day, a monthly Podcast Listener...",We send a weekly email with a theme and 5 pods...,Join the weekly #podcast designed to help you ...
descriptionUrls,[],[],[],"[{'text': 'eepurl.com/cIcBuH', 'url': 'http://...","[{'text': 'anchor.fm/shaun-donaghy', 'url': 'h..."
verified,False,False,False,False,False
created,2010-06-08T09:24:09+00:00,2016-10-19T03:35:13+00:00,2020-05-07T19:40:12+00:00,2017-01-18T22:45:04+00:00,2020-01-24T10:18:52+00:00
followersCount,2087,275,585,3222,922
friendsCount,2205,378,534,1231,1825


In [9]:
len(set(pod_rev_users.username))

317

In [10]:
pod_rev_users = pod_rev_users.drop_duplicates(subset=['username'])

In [11]:
pod_rev_users.head().T

,0,1,2,3,4
username,stephfuccio,LizWilcox79,PodRevDay,EarbudsPodCol,mindfulness_for
displayname,Stephanie Fuccio,Elizabeth Wilcox,PodRevDay,Earbuds Podcast Collective,"Shaun, Mindfulness for Beginners Podcast"
id,153349145,788584258826665984,1258481717431750656,821850926243659776,1220652146649128963
description,Freelance Podcast Editor | Podcast Host/Produc...,"Writer, Avid Role-Player, Amateur Mixologist, ...","Podcast Review Day, a monthly Podcast Listener...",We send a weekly email with a theme and 5 pods...,Join the weekly #podcast designed to help you ...
rawDescription,Freelance Podcast Editor | Podcast Host/Produc...,"Writer, Avid Role-Player, Amateur Mixologist, ...","Podcast Review Day, a monthly Podcast Listener...",We send a weekly email with a theme and 5 pods...,Join the weekly #podcast designed to help you ...
descriptionUrls,[],[],[],"[{'text': 'eepurl.com/cIcBuH', 'url': 'http://...","[{'text': 'anchor.fm/shaun-donaghy', 'url': 'h..."
verified,False,False,False,False,False
created,2010-06-08T09:24:09+00:00,2016-10-19T03:35:13+00:00,2020-05-07T19:40:12+00:00,2017-01-18T22:45:04+00:00,2020-01-24T10:18:52+00:00
followersCount,2087,275,585,3222,922
friendsCount,2205,378,534,1231,1825


# Clean User Data

In [12]:
df_users = pod_rev_users.loc[:, ['id','username', 'displayname', 'location', 'created', 'followersCount', 'friendsCount', 'url', 'verified' ]]

In [13]:
def location_extraction (df):
    '''Creates creates a geotext column to extract city and country info if possible'''

    df.loc[:, "location"] = df.loc[:, "location"].fillna("blank")
    df.loc[:, "geotext"] = df.loc[:, "location"].apply(GeoText)
    df.loc[:, 'city'] = df.loc[:, 'geotext'].apply(lambda x: x.cities)
    df.loc[:, 'country'] = df.loc[:, 'geotext'].apply(lambda x: x.countries)
    
    return df

df_users = location_extraction(df_users)

In [14]:
from geonamescache import GeonamesCache
gc = GeonamesCache()
countries = gc.get_countries()
country_info = pd.DataFrame(countries).T
country_info = country_info.set_index('geonameid').reset_index()
name_code = country_info.loc[:, ["name", "iso3"]]

us_states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
             "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA",
             "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY",
             "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX",
             "UT", "VT", "VA", "WA", "WV", "WI", "WY", "USA", "United States",
             'Seattle', "Los Angeles", "Houston", "Atlanta", "Pittsburgh"]

us_state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", 
               "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", 
               "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", 
               "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", 
               "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", 
               "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", 
               "Wisconsin", "West Virginia", "Wyoming"]

can_prov_abbrev = {'Alberta': 'AB','British Columbia': 'BC','Manitoba': 'MB', 'New Brunswick': 'NB',
                       'Newfoundland and Labrador': 'NL', 'Northwest Territories': 'NT','Nova Scotia': 'NS','Nunavut': 'NU',
                       'Ontario': 'ON','Prince Edward Island': 'PE', 'Quebec': 'QC','Saskatchewan': 'SK','Yukon': 'YT'}

can_prov_names, can_prov_abbr = zip(*can_prov_abbrev.items())

uk = ["England", 'Wales', "Scotland", 'London', "Manchester", "Isle of Wight", "Northern Ireland", "United Kingdom", 'Bailiwick of Guernsey', "UK", "Hoxton", "Jersey"]

india_city = ["Bangalore", "Delhi", "Hyderabad", "Bengaluru"]

german_city = ["Munich", "Berlin", "eisgau","Hamburg", "Dortmund"]

south_africa = ["South Africa", "Durban", "Johannesburg"]

uae = ['UAE', 'Dubai', 'Abu Dhabi']

def replacer(area, name):
    df_users.loc[(df_users.location.str.contains('|'.join(area))), "country"] = name    
    return df_users

def list_to_string(df):
    df["city"] = df['city'].apply(lambda x: "".join(map(str, x)))
    df["country"] = df['country'].apply(lambda x: "".join(map(str, x)))
    
    return df

In [15]:
def location_cleaner(df):
    df = replacer(can_prov_names, "Canada")
    df = replacer(can_prov_abbrev, "Canada")
    df = replacer(us_state_names, "United States")
    df = replacer(us_states, "United States")
    df = replacer(uk, "United Kingdom")
    df = replacer(german_city, "Germany")
    df = replacer(south_africa, "South Africa")
    df = replacer(india_city, "India")
    df = replacer(uae, 'United Arab Emirates')
    df.loc[(df.location == "Italia"), "country"] = "Italy" 
    df.loc[(df.location == "Belgrade"), "country"] = "Serbia" 
    df.loc[(df.country == "PolandSerbia"), "country"] = "Poland" 
    df = list_to_string(df)
    
    return df

In [16]:
df_users = location_cleaner(df_users)
df_users.loc[(df_users.country == "PolandSerbia"), "country"] = "Poland" 

In [17]:
df_users.country.value_counts()

                        134
United States           114
United Kingdom           33
Canada                   10
Germany                   5
Australia                 3
Poland                    3
India                     2
New Zealand               2
Malaysia                  2
Ecuador                   1
Serbia                    1
Italy                     1
France                    1
Spain                     1
Nigeria                   1
Sweden                    1
United Arab Emirates      1
Albania                   1
Name: country, dtype: int64

In [18]:
df_users_full = pd.merge(left=df_users,
                    right=name_code,
                    how='left',
                    left_on='country',
                    right_on='name')

In [19]:
df_users_full.head()

,id,username,displayname,location,created,followersCount,friendsCount,url,verified,geotext,city,country,name,iso3
0,153349145,stephfuccio,Stephanie Fuccio,Albania,2010-06-08T09:24:09+00:00,2087,2205,https://twitter.com/stephfuccio,False,<geotext.geotext.GeoText object at 0x7f157d364...,,Albania,Albania,ALB
1,788584258826665984,LizWilcox79,Elizabeth Wilcox,"Tempe, AZ",2016-10-19T03:35:13+00:00,275,378,https://twitter.com/LizWilcox79,False,<geotext.geotext.GeoText object at 0x7f157d364...,Tempe,United States,United States,USA
2,1258481717431750656,PodRevDay,PodRevDay,World,2020-05-07T19:40:12+00:00,585,534,https://twitter.com/PodRevDay,False,<geotext.geotext.GeoText object at 0x7f157d364...,,,NaN,NaN
3,821850926243659776,EarbudsPodCol,Earbuds Podcast Collective,"Los Angeles, CA",2017-01-18T22:45:04+00:00,3222,1231,https://twitter.com/EarbudsPodCol,False,<geotext.geotext.GeoText object at 0x7f157d364...,Los Angeles,United States,United States,USA
4,1220652146649128963,mindfulness_for,"Shaun, Mindfulness for Beginners Podcast",,2020-01-24T10:18:52+00:00,922,1825,https://twitter.com/mindfulness_for,False,<geotext.geotext.GeoText object at 0x7f157d364...,,,NaN,NaN


# Merge User's Geographic Data with Tweets

In [20]:
full_df = pd.merge(left=df_tweets, 
                   right=df_users_full, 
                   how='left',
                   on='username')

full_df.to_csv('data/tweets_users_october.csv', index=False)